## RFM
This technique is based on three main categories, they are:
<li><b>Recency(R)</b> – Days since last purchase</li>
<li><b>Frequency(F)</b> – Total number of purchases</li>
<li><b>Monetary Value(M)</b> – How much total money the customer spent</li>

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt

## Disable Warnings

In [2]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

## Load Data

In [ ]:
df = pd.read_excel("customer.xlsx")

In [ ]:
df

## Data Statistics

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()

In [ ]:
df1 = df

In [ ]:
df1.Country.nunique()

In [ ]:
customer_country=df1[['Country','CustomerID']].drop_duplicates()
customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False)

In [ ]:
df1 = df1.loc[df1['Country'] == 'United Kingdom']

In [ ]:
df1 = df1[pd.notnull(df1['CustomerID'])]

In [ ]:
df1 = df1[(df1['Quantity']>=0)]

In [ ]:
df1['TotalPrice'] = df1['Quantity'] * df1['UnitPrice']

In [ ]:
df1['InvoiceDate'].min()

In [ ]:
df1['InvoiceDate'].max()

In [ ]:
NOW = dt.datetime(2011,12,10)
df1['InvoiceDate'] = pd.to_datetime(df1['InvoiceDate'])

## RFM Customer Segmentation

In [ ]:
rfmTable = df1.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, 'InvoiceNo': lambda x: len(x), 'TotalPrice': lambda x: x.sum()})
rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'TotalPrice': 'monetary_value'}, inplace=True)

In [ ]:
rfmTable.head()

### Split the metrics

In [ ]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

### Create a Customer Segment RFM table

In [ ]:
segmented_rfm = rfmTable

In [ ]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

### Add segment numbers

In [ ]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()

In [ ]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

In [ ]:
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10)